In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator, Constraint)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.util import ema_logging, utilities
import time
from problem_formulation import get_model_for_problem_formulation
#from signal import signal, SIGPIPE, SIG_DFL
#signal(SIGPIPE, SIG_DFL)

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## MORDM

MORDM consists of four main steps:
1. Problem formulation 
2. Searching for candidate solutions
3. Generating an ensemble of scenarios to explore the effect of uncertainties
4. Using scenario discovery to detect the vulnerabilities of candidate solutions and to improve them


### 1. Problem formulation

For Dike Ring 1, the following objectives, all to be minimized, are of interest:\
•	A.1 Expected Annual Damage\
•	A.1 Dike Investment Costs\
•	A.1 Expected Annual Number of Deaths\
•	Total Expected Annual Number of Deaths

In [3]:
#Load in the results
base_case_results=utilities.load_results('results/base_case__17objectives_no_policy.csv')
experiments_bc, outcomes_bc = base_case_results
df_outcomes_bc_un_aggregated = pd.DataFrame(outcomes_bc)
df_outcomes_bc_un_aggregated

[MainProcess/INFO] results loaded successfully from C:\Users\User\Documents\GitHub\EPA_Group7_MBDM\final assignment\results\base_case__17objectives_no_policy.csv


,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Dike Investment Costs 2,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2
0,5.146434e+07,-1569439696,NaN,NaN,-1569436912,NaN,NaN,4,NaN,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN
1,8.246920e+08,0,0.659230,1.065912e+08,487,NaN,NaN,4,NaN,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN
2,1.025521e+09,0,0.651500,0.000000e+00,0,0.000000,0.000000e+00,4,NaN,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN
3,1.029601e+09,0,0.654764,1.079911e+07,0,0.009651,2.008020e+08,0,0.330058,0.000000e+00,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN
4,4.534010e+06,0,0.003996,2.783919e+08,0,0.285949,0.000000e+00,0,0.000000,0.000000e+00,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.327697e+09,0,0.650020,0.000000e+00,0,0.000000,3.516463e+08,0,0.438427,0.000000e+00,...,0,0.438427,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.0,0.0
996,1.015024e+09,0,0.643086,1.088708e+07,0,0.009811,3.709673e+07,0,0.063478,3.003676e+07,...,0,0.063478,1.961856e+07,0,0.011193,0.000000e+00,0,0.000000,0.0,0.0
997,3.082691e+07,0,0.016596,5.222677e+06,0,0.003833,2.834518e+07,0,0.038818,2.481569e+07,...,0,0.038818,1.913322e+07,0,0.008896,2.148592e+08,0,0.183258,0.0,0.0
998,0.000000e+00,0,0.000000,8.986093e+06,0,0.010523,1.174016e+08,0,0.250826,0.000000e+00,...,0,0.250826,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.0,0.0


In [4]:
# Aggregate over time and location to obtain the desired shape of the outcomes
def aggregate_df(df):
    df_aggregate = pd.DataFrame()
    locations = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']
    step = ['0', '1', '2']
    metrics = ['Expected Annual Damage', 'Dike Investment Costs', 'Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']
    
    for metric in metrics:
        if metric == "RfR Total Costs" or metric == "Expected Evacuation Costs":

            columns = [metric + ' ' + time for time in step]

            df_aggregate[metric] = df[columns].sum(axis=1)
        else:
            for location in locations:
                columns = [location + '_' + metric + ' ' + time for time in step]

                df_aggregate[location + '_' + metric] = df[columns].sum(axis=1)
            
            columns_locations = [location + '_' + metric for location in locations]
            df_aggregate['Total ' + metric] = df_aggregate[columns_locations].sum(axis=1)
    
 
    return df_aggregate

In [5]:
df_bc = aggregate_df(df_outcomes_bc_un_aggregated)
outcomes_of_interest = ["A.1_Expected Annual Damage","A.1_Dike Investment Costs",\
                        "A.1_Expected Number of Deaths","Total Expected Number of Deaths",\
                        "Total Expected Annual Damage"]
df_bc = df_bc[outcomes_of_interest]

In [6]:
df_bc.columns

Index(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'Total Expected Number of Deaths',
       'Total Expected Annual Damage'],
      dtype='object')

In [7]:
max_vals_dict = {}
for i in range(len(df_bc.columns)):
    max_vals_dict[df_bc.columns[i]] = max(df_bc.iloc[:,i])

In [8]:
max_vals_dict

{'A.1_Expected Annual Damage': 4037046303.204874,
 'A.1_Dike Investment Costs': 3,
 'A.1_Expected Number of Deaths': 1.9833733041503592,
 'Total Expected Number of Deaths': 5.967474269743496,
 'Total Expected Annual Damage': 6055405548.225815}

## 2. Searching for candidate solutions

In [9]:
from problem_formulation import get_model_for_problem_formulation
dike_model, planning_steps = get_model_for_problem_formulation(5)

### Constraints

For each outcome of interest we have set a specific constraint that the candidate solutions must follow. We will determine most of them by looking at the random policies that were generated with open exploration.

In [10]:
#Load in the results
results_rp=utilities.load_results('results/fully_disaggregated_open_exploration_with_policies.csv')
experiments_rp, outcomes_rp = results_rp
df_outcomes_rp_unaggregated = pd.DataFrame(outcomes_rp)

df_rp = aggregate_df(df_outcomes_rp_unaggregated)
df_rp = df_rp[outcomes_of_interest]
df_rp

[MainProcess/INFO] results loaded successfully from C:\Users\User\Documents\GitHub\EPA_Group7_MBDM\final assignment\results\fully_disaggregated_open_exploration_with_policies.csv


,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,Total Expected Number of Deaths,Total Expected Annual Damage
0,0.0,1.543459e+08,0.0,0.000000,0.000000e+00
1,0.0,2.908229e+08,0.0,0.000000,0.000000e+00
2,0.0,2.908229e+08,0.0,0.000000,0.000000e+00
3,0.0,2.908229e+08,0.0,0.000000,0.000000e+00
4,0.0,2.908229e+08,0.0,0.000000,0.000000e+00
...,...,...,...,...,...
49995,0.0,1.879048e+08,0.0,0.000411,2.460076e+06
49996,0.0,1.879048e+08,0.0,0.003782,4.752153e+07
49997,0.0,1.879048e+08,0.0,0.004289,4.181121e+07
49998,0.0,1.879048e+08,0.0,0.000723,6.631078e+06


In [11]:
constraints_dict = {}
constraints_dict[df_rp.columns[1]] = df_rp.iloc[:,1].median()
constraints_dict[df_rp.columns[4]] = df_rp.iloc[:,4].median()
constraints_dict[df_rp.columns[3]] = 0.0001 * 3 * 5

In [12]:
constraints_dict

{'A.1_Dike Investment Costs': 182429960.0293684,
 'Total Expected Annual Damage': 5092108.21861372,
 'Total Expected Number of Deaths': 0.0015}

In [13]:
max_total_deaths = 0.0001 * 3 * 5
max_total_damage = df_rp.iloc[:,4].median()
max_deaths_a1 = df_rp.iloc[:,2].median()
max_damage_a1 = df_rp.iloc[:,0].median()

In [14]:
# Adding the constraint for the allowed maximum number of deaths defined by the Delta Commission. By setting this as a constraint, the optimization will not
# return policies that do not meet this criterion.
constraints = [Constraint("Max Total Number of deaths", outcome_names=['Total Expected Number of Deaths'], function=lambda x:max(0, x-max_total_deaths)),
               #Constraint("Max Total Expected Annual Damage", outcome_names=["Total Expected Annual Damage"], function=lambda x:max(0, x-max_total_damage)),
               #Constraint("Max A.1 Expected Annual Damage", outcome_names=["A.1_Expected Annual Damage"], function=lambda x:max(0, x-max_damage_a1)),
               #Constraint("Max A.1 Expected Number of Deaths", outcome_names=["A.1_Expected Number of Deaths"], function=lambda x:max(0, x-max_deaths_a1))
              ]

In [15]:
outcome_ranges = pd.read_csv("results/outcome_ranges.csv")
outcome_ranges.set_index("Unnamed: 0",inplace=True)
outcome_ranges


,Max_values,Min_values
Unnamed: 0,,
A.1_Expected Annual Damage,3.694513e+09,0.0
A.1_Dike Investment Costs,3.753980e+08,0.0
A.1_Expected Number of Deaths,1.834596e+00,0.0
Total Expected Number of Deaths,5.276779e+00,0.0
Total Expected Annual Damage,5.736794e+09,0.0


In [ ]:
from problem_formulation_modified_2 import get_model_for_problem_formulation_modified
dike_model,  planning_steps = get_model_for_problem_formulation_modified(outcome_ranges)
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(
        nfe=20,
        searchover="levers",
        epsilons=[
            0.1,0.1,0.1,0.1,0.1
        ]
        * len(dike_model.outcomes),
        constraints=constraints,
    )

[MainProcess/INFO] pool started with 4 workers
  0%|                                                   | 0/20 [00:00<?, ?it/s]